Using optimization appraoch: predict the configuration of dolphins with a group of 3, 4, 6, 8, and 10 dolphins

In [ ]:

def optimize(group_size):
    return 

group_size = [3, 4, 6, 8, 10]
for i in group_size:
    optimize(i)
    